In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input,Dense,Concatenate, BatchNormalization, Dropout, LSTM
from keras import regularizers
from sklearn.metrics import classification_report
from keras import backend as K
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
train_neural = np.load('gdrive/MyDrive/CS626/Project/Data/train_neural_features.npy')
train_statistical = np.load('gdrive/MyDrive/CS626/Project/Data/train_statistical_features.npy')
train_external = np.load('gdrive/MyDrive/CS626/Project/Data/train_external_features.npy')

In [ ]:
print(train_neural.shape)
print(train_statistical.shape)
print(train_external.shape)

In [ ]:
stance_to_index = {'agree':0,'disagree':1,'discuss':2,'unrelated':3}
index_to_stance = {0:'agree',1:'disagree',2:'discuss',3:'unrelated'}

In [ ]:
df_train = pd.read_csv('gdrive/MyDrive/CS626/Project/Data/train_Set.csv')
Y_train = df_train[['Stance']].values.tolist()
print(len(Y_train))

In [ ]:
print(Y_train)

In [ ]:
def to_categorical(Y,num_classes):
  final_Y = np.zeros((len(Y),num_classes))
  i=0
  for stance in Y:
    final_Y[i][stance_to_index[stance[0]]] = 1
    i+=1
  return final_Y

In [ ]:
Y_train = to_categorical(Y_train, num_classes=4)

In [ ]:
print(Y_train[0])

In [ ]:
inputA = Input(shape=(4800,))
inputB = Input(shape=(10000,))
inputC = Input(shape=(60,))


# the first branch operates on the first input
x = Dense(500,activation='sigmoid',kernel_regularizer = regularizers.L2(1e-8))(inputA)
x = Dropout(0.2)(x)
x = Dense(100,activation='sigmoid')(x)
x_model = Model(inputs=inputA, outputs=x)


# the second branch opreates on the second input
y = Dense(500,activation='relu',kernel_regularizer = regularizers.L2(5e-5))(inputB)
y = Dropout(0.4)(y)
y = Dense(50,activation='relu')(y)
y_model = Model(inputs=inputB, outputs=y)

z = Dense(60, activation = "relu")(inputC)
z_model = Model(inputs = inputC, outputs = z)


# combine the output of the three branches
combined = Concatenate()([x_model.output, z_model.output, y_model.output])

# apply a FC layer and then a regression prediction on the
# combined outputs
z1 = BatchNormalization()(combined)
z1 = Dense(4, activation="softmax")(z1)


# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x_model.inputs, z_model.inputs,  y_model.inputs], outputs=z1)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(x = [train_neural,train_external,train_statistical], y = np.array(Y_train),batch_size=100,epochs=10,validation_split=0.2)

In [ ]:
import pickle
pickle.dump(model, open('gdrive/MyDrive/CS626/Project/Data/final_model.pkl', 'wb'))